In [1]:
import os 
from agno.agent import Agent 
from agno.models.openai import OpenAIChat 
from agno.tools.duckduckgo import DuckDuckGoTools 
import requests 
from dotenv import load_dotenv

In [4]:
class WeatherTool:
    def __init__(self):
        self.name = "weathertool"
        self.description = "Provides weather information for a given location."
        self.api_key = os.getenv("WEATHER_API_KEY")

    def run(self,location:str):
        try : 
            url = f"http://api.weatherapi.com/v1/current.json?key={self.api_key}&q={location}"
            response = requests.get(url)
            response.raise_for_status()  # Raise an error for bad responses
            data = response.json()
            current = data['current']
            return (f"Current weather in {location}: {current['condition']['text']}\nTemperature :{current['temp_c']}°C\n Humidity :{current['humidity']}%\nWind Speed :{current['wind_kph']} kph")
        except Exception as e :
            return f"Error fetching weather"

In [5]:
class HotelTool:
    def __init__(self):
        self.name = "hoteltool"
        self.description = "Provide the hotel recommendations for a given location."
    
    def run(self,location:str)->str:
        search_query = f"best hotel in{location}"
        search_tool = DuckDuckGoTools()
        result = search_tool.run(search_query)

In [ ]:
travel_assistant = Agent(
    model = OpenAIChat(
        id="gpt-4o",
        api_key = os.environ["OPENAI_API_KEY"],
    ),
    description = "You are an expert travel assistant helping users plan trip,recommend hotels,find the best destination and provide traveladvice",
    instructions = ["Offer detailed travel planning assistance",
        "Use WeatherTool for current weather information",
        "Use HotelTool for hotel recommendations",
        "Use DuckDuckGoTools for searching destinations and travel tips",
        "Provide concise, actionable advice in markdown format",
        "Consider budget, season, and user preferences when possible"
        
    ],
    tools = [
        WeatherTool(),
        HotelTool(),
        DuckDuckGoTools()
    ],
    markdown = True,
    memory = True
)

In [ ]:
def get_travel_help(query:str):
    travel_assistant.print_response(query,stream = True)

In [ ]:
queries = [
"What are the best places to visit in Italy for a summer trip?",
"Recommend some hotels in New York City for a weekend stay",
"What's the weather like in Tokyo, and what should I pack for a trip next week?",
"Plan a 5-day trip to Paris including top attractions and hotel suggestions"
]




In [ ]:
for q in queries:
    print(f"Query: {q}")
    get_travel_help(q)
    print("\n" + "-"*50 + "\n")
